# NATURAL LANGUAGE PROCESSING

## Setup

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/linux-64::openjdk-11.0.13-h87a67e3_0 



# MOVIES

In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

# spark = (
#     SparkSession.builder.appName("PySparkApp")
#     .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
#     .config(
#         "fs.s3a.aws.credentials.provider",
#         "com.amazonaws.auth.ContainerCredentialsProvider",
#     )
#     .getOrCreate()
# )

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b072f4f9-974d-4a9e-a0d7-5b9ae5c1bdc0;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


## Import Libraries

In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
print(f"Spark version: {spark.version}")
print(f"sparknlp version: {sparknlp.version()}")

Spark version: 3.4.0
sparknlp version: 5.1.3


## Import Data

In [5]:
%%time
bucket = "project-group34"
session = sagemaker.Session()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://project-group34/project/comments/yyyy=*


23/11/24 22:32:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 191 ms, sys: 16.4 ms, total: 207 ms
Wall time: 6.64 s


In [7]:
# comments = comments.cache()

In [6]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [7]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality").show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|    Animesuggest|      [deleted]|           [deleted]| t3_m3vnjl|gqscjse|2021-03-13 10:18:25|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| 

### DATA PROCESSING

In [8]:
# Filter out rows where 'body' or 'author' is '[deleted]'
comments_filtered = comments.filter((comments.body != '[deleted]') & (comments.author != '[deleted]'))

# Show the filtered DataFrame
comments_filtered = comments_filtered.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality")

In [9]:
comments_filtered.show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|    3|               0|
|MovieSuggestions|   alienstabler|[Holes (2003)](ht...| 

In [10]:
comments_filtered = comments_filtered.where(col("subreddit").isin("MovieSuggestions"))

In [12]:
# # Define the pipeline stages
# document_assembler = DocumentAssembler() \
#     .setInputCol("body") \
#     .setOutputCol("document")

# sentence_detector = SentenceDetector() \
#     .setInputCols(["document"]) \
#     .setOutputCol("sentence")

# tokenizer = Tokenizer() \
#     .setInputCols(["sentence"]) \
#     .setOutputCol("token")

# # Use a pretrained embeddings model, for example, BERT
# embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
#     .setInputCols(["sentence", "token"]) \
#     .setOutputCol("embeddings")

# ner_model = NerDLModel.pretrained("ner_dl_bert", "en") \
#     .setInputCols(["sentence", "token", "embeddings"]) \
#     .setOutputCol("ner")

# ner_converter = NerConverter() \
#     .setInputCols(["sentence", "token", "ner"]) \
#     .setOutputCol("ner_chunk")

# # Build the pipeline
# nlp_pipeline = Pipeline(stages=[
#     document_assembler,
#     sentence_detector,
#     tokenizer,
#     embeddings,
#     ner_model,
#     ner_converter
# ])

# # Apply the pipeline to your DataFrame
# model = nlp_pipeline.fit(comments_filtered_movies)
# result = model.transform(comments_filtered_movies)

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[OK!]


### MOVIE SUGGESTIONS EXTRACTION USING NER

In [11]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import SentenceDetector, Tokenizer, WordEmbeddingsModel, NerDLModel, NerConverter
from pyspark.ml import Pipeline

# Define the pipeline stages
document_assembler = DocumentAssembler() \
    .setInputCol("body") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

# Use GloVe embeddings
embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

# Use a lighter NER model
ner_model = NerDLModel.pretrained("ner_dl", "en") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

# Build the pipeline
nlp_pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

# Apply the pipeline to your DataFrame
model = nlp_pipeline.fit(comments_filtered)
result = model.transform(comments_filtered)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[ | ]ner_dl download started this may take some time.
Approximate size to download 13.6 MB
Download done! Loading the resource.
[ / ]

2023-11-24 22:33:23.384367: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 22:33:23.518679: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_de

[OK!]


In [13]:
result = result.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality", "ner_chunk")

In [14]:
result.show(5)

+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+
|       subreddit|        author|                body|parent_id|     id|        created_utc|score|controversiality|           ner_chunk|
+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|[{chunk, 0, 12, J...|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|t3_l20wrm|gqscozr|2021-03-13 10:21:04|    1|               0|                  []|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|t3_m3rw47|gqscpsz|2021-03-13 10:21:29|    3|               0|[{chunk, 6, 19, V...|
|MovieSuggestions|  jaymewheeler|Synchronic was co...|t3_m3rw47|gqscu76|2021-03-13 10:23:37|    2|               0|                  []|
|MovieSuggestions|    SwissBliss|Climax!!

In [15]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, ArrayType
import pyspark.sql.functions as F

# Define a UDF to filter and extract movie names
def extract_movies(chunks):
    movie_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
    return movie_names

extract_movie_names_udf = udf(extract_movies, ArrayType(StringType()))

# Apply the UDF to the DataFrame
movies_df = result.withColumn("movie_names", extract_movie_names_udf(F.col("ner_chunk")))

# # Display the results
# movies_df.select("body", "movie_names").show()

In [16]:
movies_df.show(5)

+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+------------------+
|       subreddit|        author|                body|parent_id|     id|        created_utc|score|controversiality|           ner_chunk|       movie_names|
+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|[{chunk, 0, 12, J...|                []|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|t3_l20wrm|gqscozr|2021-03-13 10:21:04|    1|               0|                  []|                []|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|t3_m3rw47|gqscpsz|2021-03-13 10:21:29|    3|               0|[{chunk, 6, 19, V...|[Vendetta\n\nPulp]|
|MovieSuggestions|  jaymewheeler|Synchronic was co...|t3_m3rw47|

In [ ]:
movies_df.write.parquet("s3a://project-group34/project/suggestions/all_movies/ner/", mode="overwrite")

### MOVIE SUGGESTIONS EXTRACTION USING REGEX

In [7]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

3.4.0


In [5]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
movies_df = spark.read.parquet("s3a://project-group34/project/suggestions/all_movies/ner/")

23/11/25 19:35:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [8]:
movies_df.show(5)

+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+------------------+
|       subreddit|        author|                body|parent_id|     id|        created_utc|score|controversiality|           ner_chunk|       movie_names|
+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|[{chunk, 0, 12, J...|                []|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|t3_l20wrm|gqscozr|2021-03-13 10:21:04|    1|               0|                  []|                []|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|t3_m3rw47|gqscpsz|2021-03-13 10:21:29|    3|               0|[{chunk, 6, 19, V...|[Vendetta\n\nPulp]|
|MovieSuggestions|  jaymewheeler|Synchronic was co...|t3_m3rw47|

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, StructType, StructField

# import spacy
import re

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # Define schema for the UDF output
# movie_schema = StructType([
#     StructField("movie_positions", ArrayType(ArrayType(StringType()))),
#     StructField("movie_names", ArrayType(StringType()))
# ])

# # UDF to extract movie names
# @udf(movie_schema)
# def extract_movie_names_udf(text):
#     doc = nlp(text)
#     movie_positions = []
#     movie_names = []

#     for ent in doc.ents:
#         if ent.label_ == "ORG" or ent.label_ == "PERSON":
#             movie_positions.append([ent.start_char, ent.end_char])
#             movie_names.append(ent.text)

#     return (movie_positions, movie_names)

# UDF to remove movie names
@udf(StringType())
def remove_movie_names_udf(text, movie_names):
    if movie_names:
        for name in movie_names:
            text = text.replace(name, ' ')
        return ' '.join(text.split())
    else:
        return text

# UDF to extract movie names using regex
@udf(ArrayType(StringType()))
def extract_movie_names_regex_udf(text, movie_names):
    movie_name_pattern = r'(?:\"([^\"]+)\"|([A-Z][a-z]*(?:\s+(?:[a-z]+\s+)*[A-Z][a-z]*)*)(?: \(\d{4}\))?)'

    movie_matches = re.findall(movie_name_pattern, text)
    movies = [match[0] or match[1] or match[2] for match in movie_matches]
    return movie_names + movies

# Remove movie names from the 'body' text
df_removed_movie_names = movies_df.withColumn("body_no_movies", remove_movie_names_udf(movies_df["body"], movies_df["movie_names"]))

# If you still want to use the regex method to supplement the NER extraction
df_final = df_removed_movie_names.withColumn("additional_movie_names", extract_movie_names_regex_udf(df_removed_movie_names["body_no_movies"], df_removed_movie_names["movie_names"]))


In [10]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "additional_movie_names")

In [11]:
df_final.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+----------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|additional_movie_names|
+----------------+--------------+--------------------+-------------------+-----+----------------+----------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|       [Jacobs Ladder]|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|               [Holes]|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|  [Vendetta\n\nPulp...|
|MovieSuggestions|  jaymewheeler|Synchronic was co...|2021-03-13 10:23:37|    2|               0|          [Synchronic]|
|MovieSuggestions|    SwissBliss|Climax!!!!! Sound...|2021-03-13 10:25:19|    2|               0|  [Climax, Sounds, ...|
+----------------+--------------

In [12]:
import nltk
#nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def remove_stop_word_from_movie_names(movies):
    # Filter out single-word movie names that are in the stop words list
    return [movie for movie in movies if not (len(movie.split()) == 1 and movie.lower() in eng_stopwords)]

remove_stop_word_udf = udf(remove_stop_word_from_movie_names, ArrayType(StringType()))

In [14]:
df_final = df_final.withColumn("movie_names_final", remove_stop_word_udf(df_final["additional_movie_names"]))

In [15]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "movie_names_final")

In [16]:
# Define a UDF to filter out single-letter movie names
def remove_single_letter_movies(movie_names):
    return [name for name in movie_names if len(name.strip()) > 1]

remove_single_letter_movies_udf = udf(remove_single_letter_movies, ArrayType(StringType()))

# Apply the UDF to the DataFrame
df_final = df_final.withColumn("movie_names_final_cleaned", remove_single_letter_movies_udf("movie_names_final"))


In [17]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "movie_names_final_cleaned")

In [18]:
df_final.show()

+----------------+-------------------+--------------------+-------------------+-----+----------------+-------------------------+
|       subreddit|             author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|
+----------------+-------------------+--------------------+-------------------+-----+----------------+-------------------------+
|MovieSuggestions|          katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|
|MovieSuggestions|       alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|
|MovieSuggestions|     ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|
|MovieSuggestions|       jaymewheeler|Synchronic was co...|2021-03-13 10:23:37|    2|               0|             [Synchronic]|
|MovieSuggestions|         SwissBliss|Climax!!!!! Sound...|2021-03-13 10:25:19|    2|            

In [19]:
df_final.write.parquet("s3a://project-group34/project/suggestions/all_movies/cleaned_with_regex/", mode="overwrite")

### EXPLODING SUGGESTIONS 

In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-683c976c-d6ab-4d7c-8705-d27f19d949ff;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
df_final = spark.read.parquet("s3a://project-group34/project/suggestions/all_movies/cleaned_with_regex/")

23/11/25 20:48:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df_final.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|
+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|
|MovieSuggestions|  jaymewheeler|Synchronic was co...|2021-03-13 10:23:37|    2|               0|             [Synchronic]|
|MovieSuggestions|    SwissBliss|Climax!!!!! Sound...|2021-03-13 10:25:19|    2|               0|     [Climax, Sounds, ...|
+-------

In [6]:
from pyspark.sql.functions import explode, col, count, split

# Flatten the movie_names column
df_flattened = df_final.withColumn("movie_name", explode(col("movie_names_final_cleaned")))

In [7]:
# List of non-movie names to be removed
non_movie_terms = [
    "Movie", "Suggestions", "Please", "Directv", "Thanks", "Editing", "Thank",
    "Netflix", "Amazon Prime Video", "Google Play Movies", "Amazon Video", "Vudu",
    "Tubi Tv", "Hoopla", "Apple Itunes", "Fandangonow", "Hulu", "Rotten Tomatoes",
    "Dailymotion", "Fubotv", "Dplay", "Showtime", "Hbo Now", "Amc On Demand",
    "Notes", "Great", "Also", "Note", "Tube", "Zz", "Db", "One", "Make", "Yes",
    "Good", "Oh", "Yeah", "Love", "P.S", "Man", "Replying", "In the comments on this post I",
    "Piracy", "Title", "Characters Minimum", "Maybe", "Fv", "OP", "Definitely",
    "Edit", "Came", "Mr", "American", "Really", "Well", "Life", "Sorry", "French",
    "But I", "Japanese", "Never", "Lol", "Watch", "House", "Link", "Like", "Dead",
    "!&lt", "Absolutely", "Mother", "Girl", "Nobody", 
    "Redbox", "M the Creator", "Metacritic", "Korean", "Dprogram",
    "Microsoft Store", "Showtime Amazon Channel", "Watched", "Hbo Now Amazon Channel",
    "IMO", "Matched", "Loved", "Still", "Seen", "Go", "List", "Let", "Best", "Day",
    "Men", "Time", "World", "Even", "Identity", "Manchester", "And I", "Devil", "Super",
    "Cbs", "Disney", "Amazing", "English", "Baby", "Blue", "Check", "Dr", "Nice",
    "Character Minimum", "Excellent", "Night", "True", "I'm", "Pretty", "Hope", "Evil",
    "Two", "Spider", "Hollywood", "Murder", "Probably", "Sling Tv", "Reddit", 
]

In [8]:
# Filter out the non-movie names and streaming services
df_flattened = df_flattened.filter(~col("movie_name").isin(non_movie_terms))

In [9]:
df_flattened.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|          movie_name|
+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|       Jacobs Ladder|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|               Holes|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|    Vendetta\n\nPulp|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|V for f

In [10]:
df_flattened.write.parquet("s3a://project-group34/project/suggestions/all_movies/flattened/", mode="overwrite")

### GROUPING SUGGESTIONS

In [27]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

3.4.0


In [28]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

In [29]:
df_flattened = spark.read.parquet("s3a://project-group34/project/suggestions/all_movies/flattened/")

In [30]:
df_flattened.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|          movie_name|
+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|       Jacobs Ladder|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|               Holes|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|    Vendetta\n\nPulp|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|V for f

In [31]:
df_movie_frequency = df_flattened.groupBy("movie_name").count()

In [32]:
df_movie_frequency.show(5)

+----------+-----+
|movie_name|count|
+----------+-----+
|       Uoc|   20|
|     Sword|   44|
|        Gb|   19|
|       Kin|   43|
|    Heaven|  425|
+----------+-----+
only showing top 5 rows



In [34]:
from pyspark.sql.functions import desc

# Sort by count in descending order
df_top_20_movies = df_movie_frequency.sort(desc('count')).limit(20)

In [36]:
df_top_20_movies.show()

+--------------+-----+
|    movie_name|count|
+--------------+-----+
|    Hereditary| 2733|
|      Parasite| 2578|
|  Interstellar| 2557|
|         Alien| 2489|
|     Prisoners| 2478|
|     The Thing| 2468|
|     Midsommar| 2446|
|       Arrival| 2375|
|    Fight Club| 2338|
|        Oldboy| 2337|
|  Blade Runner| 2317|
|  Annihilation| 2171|
|       Memento| 2159|
|     Coherence| 2011|
|     Inception| 1954|
|Shutter Island| 1928|
|           Saw| 1899|
|  Nightcrawler| 1890|
|The Lighthouse| 1829|
|           Big| 1753|
+--------------+-----+



In [37]:
df_top_20_movies_pd = df_top_20_movies.toPandas()

In [39]:
df_top_20_movies_pd.to_csv("../../data/csv/top20_movies_suggested.csv", index=False)

### MERGE WITH EXTERNAL

In [7]:
df_sentiment = spark.read.parquet("s3a://project-group34/project/sentiment_analysis/average_scores_per_movie/")

In [8]:
df_sentiment.show(5)

+--------------------+----------------------+----------------------+-----------+
|               title|average_positive_score|average_negative_score|num_reviews|
+--------------------+----------------------+----------------------+-----------+
|            The Menu|    0.8769120351379525|   0.12308796544559403|        315|
|Jim Gaffigan: Obs...|                   1.0|          7.713538E-10|          5|
|   Winter in Wartime|    0.7367684066794117|    0.2632315926923039|         67|
|                7500|    0.7694758345625764|   0.23052416603253476|        157|
|10 Things I Hate ...|     0.678140454627421|    0.3218595443279501|         77|
+--------------------+----------------------+----------------------+-----------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import col

# Constants from your dataset
C_positive = 0.7525733910082344  # Mean score of positive scores
C_negative = 0.24742646776123378 # Mean score of negative scores
m = 4  # Median number of reviews

# Adding a new column for weighted rating
positive_weighted_rating_df = df_sentiment.withColumn("weighted_rating", 
                                   (col("num_reviews") / (col("num_reviews") + m) * col("average_positive_score")) + 
                                   (m / (col("num_reviews") + m) * C_positive))

# Adding a new column for weighted rating
negative_weighted_rating_df = df_sentiment.withColumn("weighted_rating", 
                                   (col("num_reviews") / (col("num_reviews") + m) * col("average_negative_score")) + 
                                   (m / (col("num_reviews") + m) * C_negative))

In [41]:
# Get top 20 movies with highest average positive scores
top_20_positive = positive_weighted_rating_df.orderBy(desc("weighted_rating")).limit(20)


In [42]:
top_20_positive.show()

+--------------------+----------------------+----------------------+-----------+------------------+
|               title|average_positive_score|average_negative_score|num_reviews|   weighted_rating|
+--------------------+----------------------+----------------------+-----------+------------------+
|Jim Allison: Brea...|                   1.0|  8.53145166666667E-22|         48|0.9809671839237104|
|Hava Nagila (The ...|    0.9867567089655173|  0.013243287766424139|        116|0.9789505983669412|
|      Trifling Women|    0.9992484054545456|  7.515981201820878E-4|         44|0.9786921542506863|
|Blinky Bill the M...|    0.9980725209090909|  0.001927482015539...|         44|0.9776142600840194|
|Molly's Theory of...|           0.999998057|  1.942951721449868E-6|         40|0.9775049055462031|
|Blind Willow, Sle...|                   1.0|  8.522315000009178...|         32|0.9725081545564704|
|California Typewr...|          0.9936297735|  0.006370226588242396|         40|0.9717155569098395|


In [44]:
df_flattened.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|          movie_name|
+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|       Jacobs Ladder|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|               Holes|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|    Vendetta\n\nPulp|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|V for f

In [22]:
from pyspark.sql.functions import udf, col, levenshtein
from pyspark.sql.types import FloatType

In [45]:
movie_titles = [row['movie_name'] for row in df_top_20_movies.collect()]

In [46]:
movie_titles

['Hereditary',
 'Parasite',
 'Interstellar',
 'Alien',
 'Prisoners',
 'The Thing',
 'Midsommar',
 'Arrival',
 'Fight Club',
 'Oldboy',
 'Blade Runner',
 'Annihilation',
 'Memento',
 'Coherence',
 'Inception',
 'Shutter Island',
 'Saw',
 'Nightcrawler',
 'The Lighthouse',
 'Big']

In [47]:
# Filter the sentiment data DataFrame using the list
filtered_sentiment_df = positive_weighted_rating_df.filter(positive_weighted_rating_df['title'].isin(movie_titles))


In [49]:
filtered_sentiment_df.show()

+--------------+----------------------+----------------------+-----------+------------------+
|         title|average_positive_score|average_negative_score|num_reviews|   weighted_rating|
+--------------+----------------------+----------------------+-----------+------------------+
|           Big|    0.8549865664698026|   0.14501343196273178|         57| 0.848270948406749|
|Shutter Island|    0.7935697030268212|   0.20643029873821836|        259|0.7929461849733066|
|The Lighthouse|    0.8567953555230218|    0.1432046450670607|        403| 0.855771061031476|
|    Hereditary|    0.6772520104153956|    0.3227479907131384|        383|0.6780305259770788|
|       Arrival|    0.9132867864014926|   0.08671321548572934|        440|0.9118389179745263|
|         Alien|    0.6957981419639608|   0.30420185732468513|        124|0.6975723684965943|
|      Parasite|     0.825085770061366|    0.1749142296239439|       1921|0.8249350949880088|
|     Inception|    0.8771357714563982|   0.1228642294497221

In [50]:
filtered_sentiment_df_pd = filtered_sentiment_df.toPandas()

In [54]:
df_top_20_movies_pd.columns = ["title", "count"]

In [55]:
df_top_20_movies_pd

,title,count
0,Hereditary,2733
1,Parasite,2578
2,Interstellar,2557
3,Alien,2489
4,Prisoners,2478
5,The Thing,2468
6,Midsommar,2446
7,Arrival,2375
8,Fight Club,2338
9,Oldboy,2337


In [51]:
filtered_sentiment_df_pd

,title,average_positive_score,average_negative_score,num_reviews,weighted_rating
0,Big,0.854987,0.145013,57,0.848271
1,Shutter Island,0.793570,0.206430,259,0.792946
2,The Lighthouse,0.856795,0.143205,403,0.855771
3,Hereditary,0.677252,0.322748,383,0.678031
4,Arrival,0.913287,0.086713,440,0.911839
5,Alien,0.695798,0.304202,124,0.697572
6,Parasite,0.825086,0.174914,1921,0.824935
7,Inception,0.877136,0.122864,365,0.875786
8,The Thing,0.763345,0.236655,128,0.763018
9,Prisoners,0.767761,0.232239,254,0.767526


In [56]:
import pandas as pd

In [61]:
df_sentiment_of_suggestions = pd.merge(filtered_sentiment_df_pd, df_top_20_movies_pd, on="title")[["title", "weighted_rating", "count"]].sort_values(by="count", ascending=False)

In [62]:
df_sentiment_of_suggestions

,title,weighted_rating,count
3,Hereditary,0.678031,2733
6,Parasite,0.824935,2578
12,Interstellar,0.890809,2557
5,Alien,0.697572,2489
9,Prisoners,0.767526,2478
8,The Thing,0.763018,2468
13,Midsommar,0.754184,2446
4,Arrival,0.911839,2375
10,Fight Club,0.816432,2338
16,Oldboy,0.711642,2337


In [63]:
df_sentiment_of_suggestions.to_csv("../../data/csv/sentiment_of_suggestions.csv", index = False)

In [23]:
# Define a UDF to calculate normalized Levenshtein similarity score
def similarity_score(str1, str2):
    len1 = len(str1)
    len2 = len(str2)
    max_len = max(len1, len2)
    if max_len == 0:
        return 1.0
    return (max_len - levenshtein(str1, str2)) / max_len

similarity_udf = udf(similarity_score, FloatType())

In [3]:
df_top_500_movies.write.parquet(f"s3a://{bucket}/projects/comments/extracted_movies")

# ANIME

## SPARK JOB FOR NER

In [ ]:
# import sagemaker
# session = sagemaker.Session()
# bucket = "project-group34"
# !wget -qO- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/jars/spark-nlp-assembly-5.1.3.jar | aws s3 cp - s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar
# !aws s3 ls s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar

In [2]:
!mkdir -p ./code

In [8]:
%%writefile ./code/suggestion_extract_process.py

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType
import re
from pyspark.sql.functions import explode, count
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import SentenceDetector, Tokenizer, WordEmbeddingsModel, NerDLModel, NerConverter
from pyspark.ml import Pipeline

from pyspark.sql.functions import desc

import nltk
nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path", type=str, help="Path of dataset in S3")    
    parser.add_argument("--col_name_for_filtering", type=str, help="Name of the column to filter")
    args = parser.parse_args()

    spark = SparkSession.builder \
    .appName("Spark NLP")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
    
    logger.info(f"Spark version: {spark.version}")
    logger.info(f"sparknlp version: {sparknlp.version()}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path} for r/{args.col_name_for_filtering}")
    df = spark.read.parquet(args.s3_dataset_path, header=True)
    vals = [args.col_name_for_filtering]
    df_filtered = df.where(col("subreddit").isin(vals))
    logger.info(f"finished reading files...")
    logger.info(f"Number of rows in data: {df_filtered.count()}")
    
    # DATA CLEANING
    df_filtered = df_filtered.filter((df.body != '[deleted]') & (df.author != '[deleted]'))

    # Define the pipeline stages
    document_assembler = DocumentAssembler() \
        .setInputCol("body") \
        .setOutputCol("document")

    sentence_detector = SentenceDetector() \
        .setInputCols(["document"]) \
        .setOutputCol("sentence")

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    # Use GloVe embeddings
    embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en") \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("embeddings")

    # Use a lighter NER model
    ner_model = NerDLModel.pretrained("ner_dl", "en") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

    # Build the pipeline
    nlp_pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter
    ])

    # Apply the pipeline to your DataFrame
    model = nlp_pipeline.fit(df_filtered)
    result = model.transform(df_filtered)

    print("NLP Pipeline Ran Succesfully!")
                
    result = result.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality", "ner_chunk")

    # Define a UDF to filter and extract anime names
    def extract_anime(chunks):
        anime_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
        return anime_names

    extract_anime_names_udf = udf(extract_anime, ArrayType(StringType()))

    # Apply the UDF to the DataFrame
    anime_df = result.withColumn("movie_names", extract_anime_names_udf(F.col("ner_chunk")))
    
    anime_df.write.parquet("s3a://project-group34/project/suggestions/all_anime/ner/", mode="overwrite")
    
    logger.info(f"all done...")
    
if __name__ == "__main__":
    main()

Overwriting ./code/suggestion_extract_process.py


In [9]:
%%time
import boto3
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

account_id = boto3.client('sts').get_caller_identity()['Account']

CPU times: user 14.8 ms, sys: 61 µs, total: 14.9 ms
Wall time: 34.9 ms


In [10]:
account_id

'655678691473'

In [11]:
%%time
import time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    image_uri=f"{account_id}.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark:latest",
    framework_version="3.3",
    role=role,
    instance_count=8,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=7200,
)

# # S3 URI of the initialization script
# s3_uri_init_script = f's3://{bucket}/{script_key}'

# s3 paths
session = sagemaker.Session()
output_prefix_logs = f"spark_logs"

configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {"spark.executor.memory": "12g", "spark.executor.cores": "4"},
    }
]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 76.8 ms, sys: 0 ns, total: 76.8 ms
Wall time: 131 ms


In [12]:
%%time
print(f"going to extract suggestions data for subreddit=Animesuggest")
bucket = "project-group34"
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
col_name_for_filtering = "Animesuggest"

# run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
spark_processor.run(
    submit_app="./code/suggestion_extract_process.py",
    submit_jars=[f"s3://{bucket}/spark-nlp-assembly-5.1.3.jar"],
    arguments=[
        "--s3_dataset_path",
        s3_path,
        "--col_name_for_filtering",
        col_name_for_filtering,
    ],
    spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
    logs=False,
    configuration=configuration
)
# give some time for resources from this iterations to get cleaned up
# if we start the job immediately we could get insufficient resources error
time.sleep(60)

going to extract suggestions data for subreddit=Animesuggest


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-11-26-18-21-00-232


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### Import Libraries

In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
print(f"Spark version: {spark.version}")
print(f"sparknlp version: {sparknlp.version()}")

Spark version: 3.4.0
sparknlp version: 5.1.3


### Import Data

In [5]:
%%time
bucket = "project-group34"
session = sagemaker.Session()
output_prefix_data_comments = "project/comments/yyyy=2021"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://project-group34/project/comments/yyyy=2021


23/11/26 03:18:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 205 ms, sys: 8.31 ms, total: 214 ms
Wall time: 6.22 s


In [7]:
# comments = comments.cache()

In [6]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [7]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality").show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|    Animesuggest|      [deleted]|           [deleted]| t3_m3vnjl|gqscjse|2021-03-13 10:18:25|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| 

### DATA PROCESSING

In [8]:
# Filter out rows where 'body' or 'author' is '[deleted]'
comments_filtered = comments.filter((comments.body != '[deleted]') & (comments.author != '[deleted]'))

# Show the filtered DataFrame
comments_filtered = comments_filtered.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality")

In [9]:
comments_filtered.show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|    3|               0|
|MovieSuggestions|   alienstabler|[Holes (2003)](ht...| 

In [10]:
comments_filtered = comments_filtered.where(col("subreddit").isin("Animesuggest"))

In [12]:
# # Define the pipeline stages
# document_assembler = DocumentAssembler() \
#     .setInputCol("body") \
#     .setOutputCol("document")

# sentence_detector = SentenceDetector() \
#     .setInputCols(["document"]) \
#     .setOutputCol("sentence")

# tokenizer = Tokenizer() \
#     .setInputCols(["sentence"]) \
#     .setOutputCol("token")

# # Use a pretrained embeddings model, for example, BERT
# embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
#     .setInputCols(["sentence", "token"]) \
#     .setOutputCol("embeddings")

# ner_model = NerDLModel.pretrained("ner_dl_bert", "en") \
#     .setInputCols(["sentence", "token", "embeddings"]) \
#     .setOutputCol("ner")

# ner_converter = NerConverter() \
#     .setInputCols(["sentence", "token", "ner"]) \
#     .setOutputCol("ner_chunk")

# # Build the pipeline
# nlp_pipeline = Pipeline(stages=[
#     document_assembler,
#     sentence_detector,
#     tokenizer,
#     embeddings,
#     ner_model,
#     ner_converter
# ])

# # Apply the pipeline to your DataFrame
# model = nlp_pipeline.fit(comments_filtered_movies)
# result = model.transform(comments_filtered_movies)

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[OK!]


### ANIME SUGGESTIONS EXTRACTION USING NER

In [11]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import SentenceDetector, Tokenizer, WordEmbeddingsModel, NerDLModel, NerConverter
from pyspark.ml import Pipeline

# Define the pipeline stages
document_assembler = DocumentAssembler() \
    .setInputCol("body") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

# Use GloVe embeddings
embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

# Use a lighter NER model
ner_model = NerDLModel.pretrained("ner_dl", "en") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

# Build the pipeline
nlp_pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

# Apply the pipeline to your DataFrame
model = nlp_pipeline.fit(comments_filtered)
result = model.transform(comments_filtered)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[ | ]ner_dl download started this may take some time.
Approximate size to download 13.6 MB
Download done! Loading the resource.
[ / ]

2023-11-26 03:19:18.474965: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 03:19:18.607377: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_de

[OK!]


In [12]:
result = result.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality", "ner_chunk")

In [13]:
result.show(5)

+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+
|   subreddit|     author|                body| parent_id|     id|        created_utc|score|controversiality|           ner_chunk|
+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+
|Animesuggest|    Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|[{chunk, 24, 29, ...|
|Animesuggest|   Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|[{chunk, 2, 4, Im...|
|Animesuggest|    Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|[{chunk, 9, 12, T...|
|Animesuggest|Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|    3|               0|[{chunk, 4, 10, M...|
|Animesuggest|   Roboragi|**Kino no Tabi: T...|t1_gqscnqz|gqscp63|2021-03-13 10:21:

In [14]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, ArrayType
import pyspark.sql.functions as F

# Define a UDF to filter and extract movie names
def extract_anime(chunks):
    movie_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
    return movie_names

extract_anime_names_udf = udf(extract_anime, ArrayType(StringType()))

# Apply the UDF to the DataFrame
anime_df = result.withColumn("movie_names", extract_anime_names_udf(F.col("ner_chunk")))

# # Display the results
# anime_df.select("body", "movie_names").show()

In [15]:
anime_df.show(5)

+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+--------------------+
|   subreddit|     author|                body| parent_id|     id|        created_utc|score|controversiality|           ner_chunk|         movie_names|
+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+--------------------+
|Animesuggest|    Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|[{chunk, 24, 29, ...|                  []|
|Animesuggest|   Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|[{chunk, 2, 4, Im...|[Ima, Soko ni Iru...|
|Animesuggest|    Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|[{chunk, 9, 12, T...|[The Beautiful Wo...|
|Animesuggest|Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|  

In [16]:
anime_df.write.parquet("s3a://project-group34/project/suggestions/all_anime/ner/2021", mode="overwrite")

ERROR:root:KeyboardInterrupt while sending command.               (34 + 4) / 99]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## MOVIE SUGGESTIONS EXTRACTION USING REGEX

In [13]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0688283-6cc0-4f28-b5dd-e152959517e9;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [14]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

In [17]:
anime_df = spark.read.parquet("s3a://project-group34/project/suggestions/all_anime/ner/")

In [18]:
anime_df.show(5)

+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+--------------------+
|   subreddit|     author|                body| parent_id|     id|        created_utc|score|controversiality|           ner_chunk|         movie_names|
+------------+-----------+--------------------+----------+-------+-------------------+-----+----------------+--------------------+--------------------+
|Animesuggest|    Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|[{chunk, 24, 29, ...|                  []|
|Animesuggest|   Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|[{chunk, 2, 4, Im...|[Ima, Soko ni Iru...|
|Animesuggest|    Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|[{chunk, 9, 12, T...|[The Beautiful Wo...|
|Animesuggest|Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|  

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, StructType, StructField

# import spacy
import re

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # Define schema for the UDF output
# anime_schema = StructType([
#     StructField("anime_positions", ArrayType(ArrayType(StringType()))),
#     StructField("anime_names", ArrayType(StringType()))
# ])

# # UDF to extract anime names
# @udf(anime_schema)
# def extract_anime_names_udf(text):
#     doc = nlp(text)
#     anime_positions = []
#     anime_names = []

#     for ent in doc.ents:
#         if ent.label_ == "ORG" or ent.label_ == "PERSON":
#             anime_positions.append([ent.start_char, ent.end_char])
#             anime_names.append(ent.text)

#     return (anime_positions, anime_names)

# UDF to remove anime names
@udf(StringType())
def remove_anime_names_udf(text, anime_names):
    if anime_names:
        for name in anime_names:
            text = text.replace(name, ' ')
        return ' '.join(text.split())
    else:
        return text

# UDF to extract anime names using regex
@udf(ArrayType(StringType()))
def extract_anime_names_regex_udf(text, anime_names):
    anime_name_pattern = r'(?:\"([^\"]+)\"|([A-Z][a-z]*(?:\s+(?:[a-z]+\s+)*[A-Z][a-z]*)*)(?: \(\d{4}\))?)'

    anime_matches = re.findall(anime_name_pattern, text)
    anime = [match[0] or match[1] or match[2] for match in anime_matches]
    return anime_names + anime

# Remove anime names from the 'body' text
df_removed_anime_names = anime_df.withColumn("body_no_anime", remove_anime_names_udf(anime_df["body"], anime_df["movie_names"]))

# If you still want to use the regex method to supplement the NER extraction
df_final = df_removed_anime_names.withColumn("additional_anime_names", extract_anime_names_regex_udf(df_removed_anime_names["body_no_anime"], df_removed_anime_names["movie_names"]))


In [21]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "additional_anime_names")

In [22]:
df_final.show(5)

+------------+-----------+--------------------+-------------------+-----+----------------+----------------------+
|   subreddit|     author|                body|        created_utc|score|controversiality|additional_anime_names|
+------------+-----------+--------------------+-------------------+-----+----------------+----------------------+
|Animesuggest|    Athenza|{Now and Then, He...|2021-03-13 10:15:52|    2|               0|  [Now and Then, He...|
|Animesuggest|   Roboragi|**Ima, Soko ni Ir...|2021-03-13 10:16:05|    1|               0|  [Ima, Soko ni Iru...|
|Animesuggest|    Athenza|{Kino no Tabi: Th...|2021-03-13 10:20:26|    1|               0|  [The Beautiful Wo...|
|Animesuggest|Dropsoftime|Try Mahouka kouko...|2021-03-13 10:20:26|    3|               0|         [Try Mahouka]|
|Animesuggest|   Roboragi|**Kino no Tabi: T...|2021-03-13 10:21:10|    1|               0|  [Mobile Suit Gund...|
+------------+-----------+--------------------+-------------------+-----+---------------

In [23]:
import nltk
#nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

In [24]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def remove_stop_word_from_anime_names(animes):
    # Filter out single-word anime names that are in the stop words list
    return [anime for anime in animes if not (len(anime.split()) == 1 and anime.lower() in eng_stopwords)]

remove_stop_word_udf = udf(remove_stop_word_from_anime_names, ArrayType(StringType()))

In [25]:
df_final = df_final.withColumn("anime_names_final", remove_stop_word_udf(df_final["additional_anime_names"]))

In [26]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "anime_names_final")

In [27]:
# Define a UDF to filter out single-letter anime names
def remove_single_letter_animes(anime_names):
    return [name for name in anime_names if len(name.strip()) > 1]

remove_single_letter_animes_udf = udf(remove_single_letter_animes, ArrayType(StringType()))

# Apply the UDF to the DataFrame
df_final = df_final.withColumn("anime_names_final_cleaned", remove_single_letter_animes_udf("anime_names_final"))


In [28]:
df_final = df_final.select("subreddit", "author", "body", "created_utc", "score", "controversiality", "anime_names_final_cleaned")

In [29]:
df_final.show()

+------------+---------------+--------------------+-------------------+-----+----------------+-------------------------+
|   subreddit|         author|                body|        created_utc|score|controversiality|anime_names_final_cleaned|
+------------+---------------+--------------------+-------------------+-----+----------------+-------------------------+
|Animesuggest|        Athenza|{Now and Then, He...|2021-03-13 10:15:52|    2|               0|     [Now and Then, He...|
|Animesuggest|       Roboragi|**Ima, Soko ni Ir...|2021-03-13 10:16:05|    1|               0|     [Ima, Soko ni Iru...|
|Animesuggest|        Athenza|{Kino no Tabi: Th...|2021-03-13 10:20:26|    1|               0|     [The Beautiful Wo...|
|Animesuggest|    Dropsoftime|Try Mahouka kouko...|2021-03-13 10:20:26|    3|               0|            [Try Mahouka]|
|Animesuggest|       Roboragi|**Kino no Tabi: T...|2021-03-13 10:21:10|    1|               0|     [Mobile Suit Gund...|
|Animesuggest|crash-scientist|I 

In [30]:
df_final.write.parquet("s3a://project-group34/project/suggestions/all_animes/cleaned_with_regex/", mode="overwrite")

### EXPLODING SUGGESTIONS 

In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a5c57838-0382-4f2d-8997-13f5cb5114fe;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
df_final = spark.read.parquet("s3a://project-group34/project/suggestions/all_animes/cleaned_with_regex/")

23/11/26 20:54:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df_final.show(5)

+------------+-----------+--------------------+-------------------+-----+----------------+-------------------------+
|   subreddit|     author|                body|        created_utc|score|controversiality|anime_names_final_cleaned|
+------------+-----------+--------------------+-------------------+-----+----------------+-------------------------+
|Animesuggest|    Athenza|{Now and Then, He...|2021-03-13 10:15:52|    2|               0|     [Now and Then, He...|
|Animesuggest|   Roboragi|**Ima, Soko ni Ir...|2021-03-13 10:16:05|    1|               0|     [Ima, Soko ni Iru...|
|Animesuggest|    Athenza|{Kino no Tabi: Th...|2021-03-13 10:20:26|    1|               0|     [The Beautiful Wo...|
|Animesuggest|Dropsoftime|Try Mahouka kouko...|2021-03-13 10:20:26|    3|               0|            [Try Mahouka]|
|Animesuggest|   Roboragi|**Kino no Tabi: T...|2021-03-13 10:21:10|    1|               0|     [Mobile Suit Gund...|
+------------+-----------+--------------------+-----------------

In [6]:
from pyspark.sql.functions import explode, col, count, split

# Flatten the anime_names column
df_flattened = df_final.withColumn("anime_name", explode(col("anime_names_final_cleaned")))

In [16]:
from pyspark.sql.functions import when
# Rename specified anime titles
df_flattened = df_flattened.withColumn("anime_name",
                   when(col("anime_name") == "Steins", "Steins Gate")
                   .when(col("anime_name") == "Gate", "Steins Gate")
                   .when(col("anime_name") == "One", "One Piece")
                   .when(col("anime_name") == "Attack", "Attack on Titan")
                   .when(col("anime_name") == "Titan", "Attack on Titan")
                   .when(col("anime_name") == "Kaguya", "Kaguya-sama: Love Is War")
                   .when(col("anime_name") == "English: Made in Abyss", "Made in Abyss")
                   .when(col("anime_name") == "English: Violet Evergarden", "Violet Evergarden")
                   .when(col("anime_name") == "Steins; ", "Steins Gate")
                   .when(col("anime_name") == "AOT", "Attack on Titan")
                   .otherwise(col("anime_name")))

In [17]:
from pyspark.sql.functions import regexp_replace, col

# Replace "English: " and "English : " with an empty string
df_flattened = df_flattened.withColumn("anime_name", regexp_replace(col("anime_name"), "English: ?", ""))


In [18]:
from pyspark.sql.functions import trim, col

df_flattened = df_flattened.withColumn("anime_name", trim(col("anime_name")))

In [25]:
non_anime_terms = [
    "Roboragi", "Status", "Genres", "Finished", "Episodes", "English: ",
    "Animesuggest", "Edit", "Source", "Nihilate", "Synonyms", "Mistake",
    "Drama", "Comedy", "Action", "Romance", "Fantasy", "Slice of Life",
    "Supernatural", "Adventure", "Psychological", "Sci", "Fi", "Mystery",
    "Thriller", "Horror", "Thanks", "Movie", "Ecchi", "English", "Thank",
    "Manga", "Also", "Mecha", "Please", "Sports", "Releasing", "Music", "Yeah", 
    "English: The ", "Oh", "Fate", "Chapters", "OP", "Volumes", "Yes", "Episode", 
    "Maybe", "English: :", "Well", "Anime", "Season", "V Short", "Juj", "English: The",
    "Jo", "English: A", "Netflix", "Great", "", "A", ":", "on", "Really", "Like", "But I",
    "The"
]

In [26]:
# Filter out the non-anime names and streaming services
df_flattened = df_flattened.filter(~col("anime_name").isin(non_anime_terms))

In [27]:
df_flattened.show(5)

+------------+--------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|   subreddit|  author|                body|        created_utc|score|controversiality|anime_names_final_cleaned|          anime_name|
+------------+--------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|Animesuggest| Athenza|{Now and Then, He...|2021-03-13 10:15:52|    2|               0|     [Now and Then, He...|        Now and Then|
|Animesuggest| Athenza|{Now and Then, He...|2021-03-13 10:15:52|    2|               0|     [Now and Then, He...|      Here and There|
|Animesuggest|Roboragi|**Ima, Soko ni Ir...|2021-03-13 10:16:05|    1|               0|     [Ima, Soko ni Iru...|                 Ima|
|Animesuggest|Roboragi|**Ima, Soko ni Ir...|2021-03-13 10:16:05|    1|               0|     [Ima, Soko ni Iru...|    Soko ni Iru Boku|
|Animesuggest|Roboragi|**Ima, Soko ni Ir...|2021-03-13 

In [28]:
df_flattened.write.parquet("s3a://project-group34/project/suggestions/all_animes/flattened/", mode="overwrite")

### GROUPING SUGGESTIONS

In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f61bc2fb-f656-49a8-8445-7748294f048e;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
df_flattened = spark.read.parquet("s3a://project-group34/project/suggestions/all_animes/flattened/")

In [8]:
df_flattened.show(5)

+------------+----------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|   subreddit|          author|                body|        created_utc|score|controversiality|anime_names_final_cleaned|          anime_name|
+------------+----------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|Animesuggest|   ElegantTea122|Happy Sugar Life\...|2021-01-28 17:24:45|    1|               0|     [Happy Sugar Life...|Happy Sugar Life\...|
|Animesuggest|   ElegantTea122|Happy Sugar Life\...|2021-01-28 17:24:45|    1|               0|     [Happy Sugar Life...|          All have M|
|Animesuggest|fearlesslalready| {God of highschool}|2021-01-28 17:27:15|    1|               1|                    [God]|                 God|
|Animesuggest|      DuckFantic|I’ve never heard ...|2021-01-28 17:28:46|    3|               0|     [Black Clover, I’ll]|        Black Clover|

In [9]:
df_anime_frequency = df_flattened.groupBy("anime_name").count()

In [10]:
df_anime_frequency.show(5)

+--------------------+-----+
|          anime_name|count|
+--------------------+-----+
|               Sword|  565|
|Apollon (Kids on ...|    3|
|                Vivy| 2813|
|          Keiyakusha|  250|
|                 S G|  117|
+--------------------+-----+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import desc

# Sort by count in descending order
df_top_100_animes = df_anime_frequency.sort(desc('count')).limit(100)

In [12]:
df_top_100_animes.show(100, truncate=False)

+---------------------------------+-----+
|anime_name                       |count|
+---------------------------------+-----+
|Steins Gate                      |19606|
|Attack on Titan                  |16722|
|One Piece                        |10936|
|Violet Evergarden                |8780 |
|Made in Abyss                    |8558 |
|Monster                          |8543 |
|Clannad                          |7461 |
|Zero                             |6724 |
|Death Note                       |6559 |
|Code Geass                       |6169 |
|Cowboy Bebop                     |5758 |
|Horimiya                         |5710 |
|Gintama                          |5654 |
|Berserk                          |5485 |
|Dororo                           |5053 |
|Vinland Saga                     |5043 |
|Overlord                         |4887 |
|Naruto                           |4778 |
|Death Parade                     |4663 |
|Mob Psycho                       |4621 |
|Hunter x Hunter                  

In [13]:
df_top_100_animes_pd = df_top_100_animes.toPandas()

In [14]:
df_top_100_animes_pd

,anime_name,count
0,Steins Gate,19606
1,Attack on Titan,16722
2,One Piece,10936
3,Violet Evergarden,8780
4,Made in Abyss,8558
...,...,...
95,Jojo,2092
96,Asobi Asobase,2091
97,Noragami,2077
98,Orange,2070


In [15]:
df_top_100_animes_pd.to_csv("../../data/csv/top100_animes_suggested.csv", index=False)

## MERGE WITH EXTERNAL

In [7]:
df_sentiment = spark.read.parquet("s3a://project-group34/project/sentiment_analysis/average_scores_per_movie/")

In [8]:
df_sentiment.show(5)

+--------------------+----------------------+----------------------+-----------+
|               title|average_positive_score|average_negative_score|num_reviews|
+--------------------+----------------------+----------------------+-----------+
|            The Menu|    0.8769120351379525|   0.12308796544559403|        315|
|Jim Gaffigan: Obs...|                   1.0|          7.713538E-10|          5|
|   Winter in Wartime|    0.7367684066794117|    0.2632315926923039|         67|
|                7500|    0.7694758345625764|   0.23052416603253476|        157|
|10 Things I Hate ...|     0.678140454627421|    0.3218595443279501|         77|
+--------------------+----------------------+----------------------+-----------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import col

# Constants from your dataset
C_positive = 0.7525733910082344  # Mean score of positive scores
C_negative = 0.24742646776123378 # Mean score of negative scores
m = 4  # Median number of reviews

# Adding a new column for weighted rating
positive_weighted_rating_df = df_sentiment.withColumn("weighted_rating", 
                                   (col("num_reviews") / (col("num_reviews") + m) * col("average_positive_score")) + 
                                   (m / (col("num_reviews") + m) * C_positive))

# Adding a new column for weighted rating
negative_weighted_rating_df = df_sentiment.withColumn("weighted_rating", 
                                   (col("num_reviews") / (col("num_reviews") + m) * col("average_negative_score")) + 
                                   (m / (col("num_reviews") + m) * C_negative))

In [41]:
# Get top 20 movies with highest average positive scores
top_20_positive = positive_weighted_rating_df.orderBy(desc("weighted_rating")).limit(20)


In [42]:
top_20_positive.show()

+--------------------+----------------------+----------------------+-----------+------------------+
|               title|average_positive_score|average_negative_score|num_reviews|   weighted_rating|
+--------------------+----------------------+----------------------+-----------+------------------+
|Jim Allison: Brea...|                   1.0|  8.53145166666667E-22|         48|0.9809671839237104|
|Hava Nagila (The ...|    0.9867567089655173|  0.013243287766424139|        116|0.9789505983669412|
|      Trifling Women|    0.9992484054545456|  7.515981201820878E-4|         44|0.9786921542506863|
|Blinky Bill the M...|    0.9980725209090909|  0.001927482015539...|         44|0.9776142600840194|
|Molly's Theory of...|           0.999998057|  1.942951721449868E-6|         40|0.9775049055462031|
|Blind Willow, Sle...|                   1.0|  8.522315000009178...|         32|0.9725081545564704|
|California Typewr...|          0.9936297735|  0.006370226588242396|         40|0.9717155569098395|


In [44]:
df_flattened.show(5)

+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|       subreddit|        author|                body|        created_utc|score|controversiality|movie_names_final_cleaned|          movie_name|
+----------------+--------------+--------------------+-------------------+-----+----------------+-------------------------+--------------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|2021-03-13 10:19:07|    2|               0|          [Jacobs Ladder]|       Jacobs Ladder|
|MovieSuggestions|  alienstabler|[Holes (2003)](ht...|2021-03-13 10:21:04|    1|               0|                  [Holes]|               Holes|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|    Vendetta\n\nPulp|
|MovieSuggestions|ThalesHedonist|V for Vendetta\n\...|2021-03-13 10:21:29|    3|               0|     [Vendetta\n\nPulp...|V for f

In [22]:
from pyspark.sql.functions import udf, col, levenshtein
from pyspark.sql.types import FloatType

In [45]:
movie_titles = [row['movie_name'] for row in df_top_20_movies.collect()]

In [46]:
movie_titles

['Hereditary',
 'Parasite',
 'Interstellar',
 'Alien',
 'Prisoners',
 'The Thing',
 'Midsommar',
 'Arrival',
 'Fight Club',
 'Oldboy',
 'Blade Runner',
 'Annihilation',
 'Memento',
 'Coherence',
 'Inception',
 'Shutter Island',
 'Saw',
 'Nightcrawler',
 'The Lighthouse',
 'Big']

In [47]:
# Filter the sentiment data DataFrame using the list
filtered_sentiment_df = positive_weighted_rating_df.filter(positive_weighted_rating_df['title'].isin(movie_titles))


In [49]:
filtered_sentiment_df.show()

+--------------+----------------------+----------------------+-----------+------------------+
|         title|average_positive_score|average_negative_score|num_reviews|   weighted_rating|
+--------------+----------------------+----------------------+-----------+------------------+
|           Big|    0.8549865664698026|   0.14501343196273178|         57| 0.848270948406749|
|Shutter Island|    0.7935697030268212|   0.20643029873821836|        259|0.7929461849733066|
|The Lighthouse|    0.8567953555230218|    0.1432046450670607|        403| 0.855771061031476|
|    Hereditary|    0.6772520104153956|    0.3227479907131384|        383|0.6780305259770788|
|       Arrival|    0.9132867864014926|   0.08671321548572934|        440|0.9118389179745263|
|         Alien|    0.6957981419639608|   0.30420185732468513|        124|0.6975723684965943|
|      Parasite|     0.825085770061366|    0.1749142296239439|       1921|0.8249350949880088|
|     Inception|    0.8771357714563982|   0.1228642294497221

In [50]:
filtered_sentiment_df_pd = filtered_sentiment_df.toPandas()

In [54]:
df_top_20_movies_pd.columns = ["title", "count"]

In [55]:
df_top_20_movies_pd

,title,count
0,Hereditary,2733
1,Parasite,2578
2,Interstellar,2557
3,Alien,2489
4,Prisoners,2478
5,The Thing,2468
6,Midsommar,2446
7,Arrival,2375
8,Fight Club,2338
9,Oldboy,2337


In [51]:
filtered_sentiment_df_pd

,title,average_positive_score,average_negative_score,num_reviews,weighted_rating
0,Big,0.854987,0.145013,57,0.848271
1,Shutter Island,0.793570,0.206430,259,0.792946
2,The Lighthouse,0.856795,0.143205,403,0.855771
3,Hereditary,0.677252,0.322748,383,0.678031
4,Arrival,0.913287,0.086713,440,0.911839
5,Alien,0.695798,0.304202,124,0.697572
6,Parasite,0.825086,0.174914,1921,0.824935
7,Inception,0.877136,0.122864,365,0.875786
8,The Thing,0.763345,0.236655,128,0.763018
9,Prisoners,0.767761,0.232239,254,0.767526


In [56]:
import pandas as pd

In [61]:
df_sentiment_of_suggestions = pd.merge(filtered_sentiment_df_pd, df_top_20_movies_pd, on="title")[["title", "weighted_rating", "count"]].sort_values(by="count", ascending=False)

In [62]:
df_sentiment_of_suggestions

,title,weighted_rating,count
3,Hereditary,0.678031,2733
6,Parasite,0.824935,2578
12,Interstellar,0.890809,2557
5,Alien,0.697572,2489
9,Prisoners,0.767526,2478
8,The Thing,0.763018,2468
13,Midsommar,0.754184,2446
4,Arrival,0.911839,2375
10,Fight Club,0.816432,2338
16,Oldboy,0.711642,2337


In [63]:
df_sentiment_of_suggestions.to_csv("../../data/csv/sentiment_of_suggestions.csv", index = False)

In [23]:
# Define a UDF to calculate normalized Levenshtein similarity score
def similarity_score(str1, str2):
    len1 = len(str1)
    len2 = len(str2)
    max_len = max(len1, len2)
    if max_len == 0:
        return 1.0
    return (max_len - levenshtein(str1, str2)) / max_len

similarity_udf = udf(similarity_score, FloatType())

In [3]:
df_top_500_movies.write.parquet(f"s3a://{bucket}/projects/comments/extracted_movies")

# SPARK JOB

In [ ]:
# import sagemaker
# session = sagemaker.Session()
# bucket = "project-group34"
# !wget -qO- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/jars/spark-nlp-assembly-5.1.3.jar | aws s3 cp - s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar
# !aws s3 ls s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar

In [2]:
!mkdir -p ./code

In [29]:
%%writefile ./code/suggestion_extract_process.py

import subprocess
import sys

import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(sys.path)

subprocess.check_call([sys.executable, "-m", "pip", "install", "spark-nlp"])

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType
import re
from pyspark.sql.functions import explode, count
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

from pyspark.sql.functions import desc

import nltk
nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path", type=str, help="Path of dataset in S3")    
    parser.add_argument("--col_name_for_filtering", type=str, help="Name of the column to filter")
    args = parser.parse_args()

    spark = SparkSession.builder \
    .appName("Spark NLP")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
    
    logger.info(f"Spark version: {spark.version}")
    logger.info(f"sparknlp version: {sparknlp.version()}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path} for r/{args.col_name_for_filtering}")
    df = spark.read.parquet(args.s3_dataset_path, header=True)
    vals = [args.col_name_for_filtering]
    df_filtered = df.where(col("subreddit").isin(vals))
    logger.info(f"finished reading files...")
    
    # DATA CLEANING
    comments_filtered = df_filtered.filter((df.body != '[deleted]') & (df.author != '[deleted]'))
    comments_filtered_movies = comments_filtered.where(col("subreddit").isin("MovieSuggestions"))

    # Define the pipeline stages
    document_assembler = DocumentAssembler() \
        .setInputCol("body") \
        .setOutputCol("document")

    sentence_detector = SentenceDetector() \
        .setInputCols(["document"]) \
        .setOutputCol("sentence")

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    # Use a pretrained embeddings model, for example, BERT
    embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("embeddings")

    ner_model = NerDLModel.pretrained("ner_dl_bert", "en") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

    # Build the pipeline
    nlp_pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter
    ])

    # Apply the pipeline to your DataFrame
    model = nlp_pipeline.fit(comments_filtered_movies)
    result = model.transform(comments_filtered_movies)
    
    print("NLP Pipeline Ran Succesfully!")

    # Define a UDF to filter and extract movie names
    def extract_movies(chunks):
        movie_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
        return movie_names

    extract_movie_names_udf = udf(extract_movies, ArrayType(StringType()))

    # Apply the UDF to the DataFrame
    movies_df = result.withColumn("movie_names", extract_movie_names_udf(F.col("ner_chunk")))


    @udf(StringType())
    def remove_movie_names_udf(text, movie_names):
        if movie_names:
            for name in movie_names:
                text = text.replace(name, ' ')
            return ' '.join(text.split())
        else:
            return text

    # UDF to extract movie names using regex
    @udf(ArrayType(StringType()))
    def extract_movie_names_regex_udf(text, movie_names):
        movie_name_pattern = r'(?:\"([^\"]+)\"|([A-Z][a-z]*(?:\s+(?:[a-z]+\s+)*[A-Z][a-z]*)*)(?: \(\d{4}\))?)'

        movie_matches = re.findall(movie_name_pattern, text)
        movies = [match[0] or match[1] or match[2] for match in movie_matches]
        return movie_names + movies
    
    def remove_stop_word_from_movie_names(movies):
        if movies and len(movies[0].split()) == 1 and movies[0].lower() in eng_stopwords:
            return movies[1:]
        return movies

    # Remove movie names from the 'body' text
    df_removed_movie_names = movies_df.withColumn("body_no_movies", remove_movie_names_udf(movies_df["body"], movies_df["movie_names"]))

    # The regex method to supplement the NER extraction
    df_final = df_removed_movie_names.withColumn("additional_movie_names", extract_movie_names_regex_udf(df_removed_movie_names["body_no_movies"], df_removed_movie_names["movie_names"]))

    df_final = df_final.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality", "additional_movie_names")
    
    print("Movie Names Extracted")
    
    remove_stop_word_udf = udf(remove_stop_word_from_movie_names, ArrayType(StringType()))

    df_final = df_final.withColumn("movie_names_final", remove_stop_word_udf(df_final["additional_movie_names"]))
    
    # Flatten the movie_names column
    df_flattened = df_final.withColumn("movie_name", explode(col("movie_names_final")))

    # Group by movie_name and count the occurrences
    df_frequency = df_flattened.groupBy("movie_name").agg(count("*").alias("frequency"))
    
    print("Aggregation Done!")
    
    # Sort the DataFrame by frequency in descending order and take the top 1000
    df_top_1000_movies = df_frequency.orderBy(desc("frequency")).limit(1000)
    
    # df_top_1000_movies_pd = df_top_1000_movies.toPandas()
    
    bucket = "project-group34"
    output_prefix_data_comments = "project/comments/"
    s3_path = f"s3a://{bucket}/{output_prefix_data_comments}" + args.col_name_for_filtering + "/"
    
    print(f"Writing to {s3_path}")
    df_top_1000_movies.write.parquet(s3_path)
    # Save the DataFrame in CSV format to S3
    #df_top_1000_movies.write.option("header", "true").mode("overwrite").csv(s3_path)
    print(f"Finished writing to {s3_path}")

    # df_top_1000_movies.to_csv(f"{s3_path}/movie_suggestions/{csv_name}")
    
    logger.info(f"all done...")
    
if __name__ == "__main__":
    main()

Overwriting ./code/suggestion_extract_process.py


In [10]:
%%time
import boto3
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

account_id = boto3.client('sts').get_caller_identity()['Account']

CPU times: user 19.3 ms, sys: 317 µs, total: 19.6 ms
Wall time: 41.5 ms


In [11]:
account_id

'655678691473'

In [30]:
%%time
import time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    image_uri=f"{account_id}.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark:latest",
    framework_version="3.3",
    role=role,
    instance_count=8,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=7200,
)

# # S3 URI of the initialization script
# s3_uri_init_script = f's3://{bucket}/{script_key}'

# s3 paths
session = sagemaker.Session()
output_prefix_logs = f"spark_logs"

configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {"spark.executor.memory": "12g", "spark.executor.cores": "4"},
    }
]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 110 ms, sys: 50 µs, total: 110 ms
Wall time: 203 ms


In [2]:
%%time
subreddit_list = ["MovieSuggestions", "televisionsuggestions", "Animesuggest"]
for subreddit in subreddit_list:
    print(f"going to extract suggestions data for subreddit={subreddit}")
    bucket = "project-group34"
    output_prefix_data_comments = "project/comments/yyyy=*"
    s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
    col_name_for_filtering = subreddit

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/suggestion_extract_process.py",
        submit_jars=[f"s3://{bucket}/spark-nlp-assembly-5.1.3.jar"],
        arguments=[
            "--s3_dataset_path",
            s3_path,
            "--col_name_for_filtering",
            col_name_for_filtering,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)